In [1]:
%%writefile app.py
import streamlit as st
import pandas as pd
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from urllib.parse import parse_qs
import numpy as np

# -------------------- CARGA DE DATOS --------------------
@st.cache_resource
def load_data():
    df = pd.read_csv("RepChecaLimp3.csv")
    numeric_df = df.select_dtypes(['float', 'int'])
    numeric_cols = numeric_df.columns
    text_df = df.select_dtypes(['object'])
    text_cols = text_df.columns
    return df, numeric_cols, text_cols, numeric_df

df, numeric_cols, text_cols, numeric_df = load_data()

# -------------------- CSS --------------------
st.markdown("""
<style>
body {
    background-color: #001a00;
    color: #00ffcc;
    font-family: 'Orbitron', sans-serif;
}

div[data-testid="stSidebar"] {
    background-color: #000000;
    border-right: 2px solid #00ffcc;
}

h1, h2, h3 {
    color: #00ffcc;
    font-family: "Orbitron", sans-serif;
}

hr {
    border-top: 2px solid #00ffff;
}

.button-container {
    display: flex;
    justify-content: space-around;
    flex-wrap: wrap;
    margin-bottom: 1.5em;
}

.cyber-btn {
    font-weight: bold;
    padding: 0.6em 1.2em;
    font-size: 1em;
    border-radius: 10px;
    border: none;
    cursor: pointer;
    transition: 0.3s ease;
    text-shadow: 0 0 2px #000;
    display: inline-block;
    text-align: center;
    text-decoration: none;
    color: black;
    margin: 0.2em;
}

.btn-view1 { background-color: #ff66cc; }
.btn-view1:hover { box-shadow: 0 0 15px #ff66cc; }

.btn-view2 { background-color: #0099ff; }
.btn-view2:hover { box-shadow: 0 0 15px #0099ff; }

.btn-view3 { background-color: #cc00ff; }
.btn-view3:hover { box-shadow: 0 0 15px #cc00ff; }

.btn-view4 { background-color: #ff9900; }
.btn-view4:hover { box-shadow: 0 0 15px #ff9900; }

.btn-view5 { background-color: #66ff66; }
.btn-view5:hover { box-shadow: 0 0 15px #66ff66; }

.btn-view6 { background-color: #00ffff; }
.btn-view6:hover { box-shadow: 0 0 15px #00ffff; }

.btn-view7 { background-color: #ff0066; }
.btn-view7:hover { box-shadow: 0 0 15px #ff0066; }
</style>
""", unsafe_allow_html=True)

# -------------------- TÍTULO --------------------
st.title("🛸 - República Checa")
st.subheader("🧭 Selecciona una opcion:")

# -------------------- BOTONES DE NAVEGACIÓN --------------------
st.markdown("""
<div class="button-container">
    <a href="/?view=view1" class="cyber-btn btn-view1">🔍 Univariado 1</a>
    <a href="/?view=view2" class="cyber-btn btn-view2">🛰 Dispersion 2</a>
    <a href="/?view=view3" class="cyber-btn btn-view3">🌌 Pastel 3</a>
    <a href="/?view=view4" class="cyber-btn btn-view4">🤖 Barras 4</a>
    <a href="/?view=view5" class="cyber-btn btn-view5">🧠 Heat Map 5</a>
    <a href="/?view=view6" class="cyber-btn btn-view6">📉 Regresion Lineal 6</a>
    <a href="/?view=view7" class="cyber-btn btn-view7">📊 Regresion Multiple 7</a>
</div>
""", unsafe_allow_html=True)

st.markdown("---")

# -------------------- CONTROLADOR DE VISTA --------------------
query_params = st.experimental_get_query_params()
view = query_params.get("view", ["view1"])[0]

# -------------------- VISTAS --------------------

# View 1: Análisis Univariado
if view == "view1":
    st.header("🔍 Análisis Univariado")
    st.sidebar.title("🎛 Panel de Control")
    st.sidebar.subheader("📊 Explora los datos")

    if st.sidebar.checkbox("📄 Mostrar dataset completo"):
        st.write(df)

    variable = st.sidebar.selectbox("📌 Selecciona variable numérica", options=numeric_cols)
    fig = px.histogram(df, x=variable, nbins=20, title=f'Distribución de {variable}')
    st.plotly_chart(fig)
    st.markdown("📈 Descripción estadística:")
    st.write(df[variable].describe())

# View 2: Dispersión
elif view == "view2":
    st.header("🛰 Dispersión de Variables")
    x_var = st.sidebar.selectbox("🧩 Variable X", options=numeric_cols)
    y_var = st.sidebar.selectbox("🔧 Variable Y", options=numeric_cols)
    fig2 = px.scatter(df, x=x_var, y=y_var, title=f'Dispersión entre {x_var} y {y_var}')
    st.plotly_chart(fig2)

# View 3: Gráfico Circular
elif view == "view3":
    st.header("🌌 Gráfico Circular")
    cat_var = st.sidebar.selectbox("🧬 Variable Categórica", options=text_cols)
    num_var = st.sidebar.selectbox("📐 Variable Numérica", options=numeric_cols)
    fig3 = px.pie(df, names=cat_var, values=num_var, title=f'Proporción de {num_var} por {cat_var}')
    st.plotly_chart(fig3)

# View 4: Barras
elif view == "view4":
    st.header("🤖 Gráfico de Barras")
    cat_var = st.sidebar.selectbox("🔤 Variable Categórica", options=text_cols)
    num_var = st.sidebar.selectbox("🧮 Variable Numérica", options=numeric_cols)
    fig4 = px.bar(df, x=cat_var, y=num_var, title=f'{num_var} por {cat_var}')
    st.plotly_chart(fig4)

# View 5: Mapa de calor personalizado
elif view == "view5":
    st.header("🧠 Mapa de Calor de Correlaciones")
    selected_vars = st.multiselect("🧠 Selecciona variables numéricas", options=numeric_cols, default=list(numeric_cols[:5]))

    if len(selected_vars) >= 2:
        fig, ax = plt.subplots(figsize=(10, 8))
        sns.heatmap(df[selected_vars].corr(), annot=True, cmap="Purples", fmt=".2f", ax=ax)
        st.pyplot(fig)
    else:
        st.warning("⚠ Selecciona al menos dos variables para generar el mapa de calor.")

# View 6: Regresión Lineal Simple
elif view == "view6":
    st.header("📉 Regresión Lineal Simple")
    x_var = st.sidebar.selectbox("📍 Variable independiente (X)", options=numeric_cols)
    y_var = st.sidebar.selectbox("🎯 Variable dependiente (Y)", options=numeric_cols)

    X = df[[x_var]]
    y = df[y_var]

    model = LinearRegression()
    model.fit(X, y)
    y_pred = model.predict(X)

    r = np.corrcoef(df[x_var], df[y_var])[0, 1]
    r_squared = model.score(X, y)

    fig = px.scatter(df, x=x_var, y=y_var, trendline="ols", title=f'Regresión Lineal: {x_var} vs {y_var}')
    st.plotly_chart(fig)

    st.markdown(f"""
    *🧮 Ecuación:*  
    y = {model.coef_[0]:.2f} * x + {model.intercept_:.2f}  
    
    *📊 R² (Coef. de determinación):* {r_squared:.4f}  
    *🔗 r (Coef. de correlación):* {r:.4f}
    """)

# View 7: Regresión Lineal Múltiple
elif view == "view7":
    st.header("📊 Regresión Lineal Múltiple")
    y_var = st.sidebar.selectbox("🎯 Variable dependiente (Y)", options=numeric_cols)
    x_vars = st.sidebar.multiselect("📍 Variables independientes (X)", options=[col for col in numeric_cols if col != y_var])

    if x_vars:
        X = df[x_vars]
        y = df[y_var]

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        model = LinearRegression()
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        r_squared = model.score(X_test, y_test)
        r = np.corrcoef(y_test, y_pred)[0, 1]

        st.markdown(f"📊 R² (Coef. de determinación en test):** {r_squared:.4f}")
        st.markdown(f"🔗 r (Coef. de correlación en test):** {r:.4f}")

        st.markdown("🧮 Coeficientes:")
        coef_df = pd.DataFrame({
            "Variable": x_vars,
            "Coeficiente": model.coef_
        })
        st.write(coef_df)

        result_df = pd.DataFrame({"Real": y_test, "Predicho": y_pred}).reset_index(drop=True)
        fig = px.line(result_df, title="📈 Comparación entre valores reales y predichos")
        st.plotly_chart(fig)

Overwriting app.py


In [2]:
! npm install localtunnel


up to date, audited 23 packages in 1s

3 packages are looking for funding
  run `npm fund` for details

2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
